In [17]:
module ResetMclWorld

using Plots
using Distributions
using StatsBase
using LinearAlgebra
using Printf
using Logging
Logging.disable_logging(Logging.Info)

include("../scripts/mcl.jl")
import .MclWorld: RealRobot, RealCamera
import .MclWorld: IdealAgent, Agent, EstimationAgent, Landmark, Map, Camera, Robot, World
import .MclWorld: data, decision, state_transition, circle_shape, draw, append
import .MclWorld: Estimator, Particle, set_ml, motion_update, observation_update, resampling_system

mutable struct ResetMcl <: Estimator
    init_pose::Array{Float64}
    num::Int64
    map::Map
    particles::Array{Particle}
    distance_dev_rate::Float64
    direction_dev::Float64
    motion_noise_rate_pdf::MvNormal
    ml::Particle
    pose::Array{Float64}
    alphas::Dict

    function ResetMcl(
        init_pose,
        num;
        map=Map(),
        particles=[Particle(init_pose, 1.0)],
        distance_dev_rate=0.14,
        direction_dev=0.05,
        motion_noise_stds=Dict([("nn", 0.19), ("no", 0.001), ("on", 0.13), ("oo", 0.2)]),
        ml = particles[1],
        pose = ml.pose,
        alphas=Dict()
        )
    
        c = diagm([motion_noise_stds["nn"]^2, motion_noise_stds["no"]^2, motion_noise_stds["on"]^2, motion_noise_stds["oo"]^2])
        motion_noise_rate_pdf = MvNormal([0, 0, 0, 0], c)
    
        new(
            init_pose,
            num,
            map,
            particles,
            distance_dev_rate,
            direction_dev,
            motion_noise_rate_pdf,
            ml,
            pose,
            alphas
        )
    end

end

function observation_update(self::Estimator, observation)
    for p in self.particles
        observation_update(p, observation, self.map, self.distance_dev_rate, self.direction_dev)
    end
    
    #alpha値の記録
    alpha = sum([p.weight for p in self.particles])
    obsnum = length(observation)
    if !haskey(self.alphas, obsnum)
        self.alphas[obsnum] = []
    end
    push!(self.alphas[obsnum], alpha)
    
    set_ml(self)
    resampling_system(self)
end

end

Main.ResetMclWorld

In [20]:
using .ResetMclWorld
using Distributions

function trial(animation=true)

    time_interval = 0.1    
    world = ResetMclWorld.World(30, 0.1)

    ### 地図を生成して2つランドマークを追加 ###
    m = ResetMclWorld.Map()
    ResetMclWorld.append(m, ResetMclWorld.Landmark([-4, 2]))
    ResetMclWorld.append(m, ResetMclWorld.Landmark([2, -3]))
    ResetMclWorld.append(m, ResetMclWorld.Landmark([3, 3]))
    ResetMclWorld.append(world, m)

    initial_pose = [0.0, 0.0, 0.0]
    pf = ResetMclWorld.ResetMcl(initial_pose, 100, map=m)
    a = ResetMclWorld.EstimationAgent(time_interval, 0.2, 10.0/180.0*pi, pf)
    r = ResetMclWorld.RealRobot(initial_pose, :red, 0.2, a, sensor=ResetMclWorld.RealCamera(m))
    ResetMclWorld.append(world, r)

    if animation
        ResetMclWorld.draw(world)
    else
        ResetMclWorld.draw(world)
        return pf
    end
end

pf = trial(false)

Main.ResetMclWorld.ResetMcl([0.0, 0.0, 0.0], 100, Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Map(Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark[Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark([-4.0, 2.0], 1), Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark([2.0, -3.0], 2), Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark([3.0, 3.0], 3)]), Main.ResetMclWorld.MclWorld.Particle[Main.ResetMclWorld.MclWorld.Particle([-0.7382751895703321, 0.5215779769855301, 5.190680636832354], 1.0)], 0.14, 0.05, FullNormal(
dim: 4
μ: [0.0, 0.0, 0.0, 0.0]
Σ: [0.0361 0.0 0.0 0.0; 0.0 1.0e-6 0.0 0.0; 0.0 0.0 0.016900000000000002 0.0; 0.0 0.0 0.0 0.04000000000000001]
)
, Main.ResetMclWorld.MclWorld.Particle([-0.7382751895703321, 0.5215779769855301, 5.190680636832354], 1.0), [-0.7382751895703321, 0.5215779769855301, 5.190680636832354], Dict{Any, Any}(0 => Any[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1

In [27]:
for (key, val) in pf.alphas ###mclalpharesult
    println("landmarks: ", key, " particles: ", length(pf.particles), " min: ", minimum(pf.alphas[key]), " max: ", maximum(pf.alphas[key]))
end

landmarks: 0 particles: 1 min: 1.0 max: 1.0
landmarks: 2 particles: 1 min: 4.4855636744425915 max: 32.309800973410155
landmarks: 1 particles: 1 min: 4.744580591880371e-45 max: 6.548231799610043
